In [1]:
# Dependencies
import pandas as pd
import numpy as np

## Extraction

In [2]:
income_df = pd.read_excel('../Resources/neighbourhood-income-data-2011.xlsx')
income_df.head()

Category       Topic                                   Attribute  \
0  Population  Population                            Population, 2011   
1  Population  Population                            Population, 2006   
2  Population  Population  Population percentage change, 2006 to 2011   
3  Population  Population     Population density per square kilometre   
4  Population   Dwellings                     Total private dwellings   

   City of Toronto  Agincourt North  Agincourt South-Malvern West  Alderwood  \
0        2615060.0          30279.0                       21988.0    11904.0   
1        2503281.0          30156.0                       21562.0    11656.0   
2              4.5              NaN                           NaN        NaN   
3           4149.5              NaN                           NaN        NaN   
4        1107851.0           9341.0                        7861.0     4840.0   

     Annex  Banbury-Don Mills  Bathurst Manor  ...  Willowdale West  \
0  29177.0            26918.0         15434.0  ...          15004.0   
1  27482.0            25439.0         14945.0  ...          12517.0   
2      NaN                NaN             NaN  ...              NaN   
3      NaN                NaN             NaN  ...              NaN   
4  17172.0            12118.0          6320.0  ...           6931.0   

   Willowridge-Martingrove-Richview   Woburn  Woodbine Corridor  \
0                           21343.0  53350.0            11703.0   
1                           20907.0  52461.0            11550.0   
2                               NaN      NaN                NaN   
3                               NaN      NaN                NaN   
4                            8336.0  19181.0             5391.0   

   Woodbine-Lumsden  Wychwood  Yonge-Eglinton  Yonge-St.Clair  \
0            7826.0   13986.0         10578.0         11652.0   
1            8051.0   14194.0         10497.0         11235.0   
2               NaN       NaN             NaN             NaN   
3               NaN       NaN             NaN             NaN   
4            3645.0    6002.0          5550.0          7128.0   

   York University Heights  Yorkdale-Glen Park  
0                  27713.0             14687.0  
1                  26140.0             14830.0  
2                      NaN                 NaN  
3                      NaN                 NaN  
4                  11722.0              5444.0  

[5 rows x 144 columns]

## Transform

In [3]:
income_df = income_df.loc[(income_df["Category"] == 'Income') & \
                          (income_df["Topic"]  == 'Income of households') & \
                          ((income_df["Attribute"] == 'Median household total income $')| \
                           (income_df["Attribute"]  == 'Average household total income $'))]
income_df.head()

Category                 Topic                         Attribute  \
755   Income  Income of households   Median household total income $   
756   Income  Income of households  Average household total income $   

     City of Toronto  Agincourt North  Agincourt South-Malvern West  \
755          58381.0          60162.0                       59013.0   
756          87038.0          73512.0                       73156.0   

     Alderwood     Annex  Banbury-Don Mills  Bathurst Manor  ...  \
755    71585.0   57825.0            70039.0         56563.0  ...   
756    85206.0  114333.0           115944.0         76823.0  ...   

     Willowdale West  Willowridge-Martingrove-Richview   Woburn  \
755          60537.0                           63701.0  52018.0   
756          82203.0                           81035.0  63651.0   

     Woodbine Corridor  Woodbine-Lumsden  Wychwood  Yonge-Eglinton  \
755            72937.0           65259.0   55941.0         72977.0   
756            88615.0           75872.0   84199.0        126230.0   

     Yonge-St.Clair  York University Heights  Yorkdale-Glen Park  
755         71728.0                  45534.0             54093.0  
756        153993.0                  55531.0             72429.0  

[2 rows x 144 columns]

In [4]:
# Did transpose of all cities columns to 1 single column

transposed_df = income_df.loc[:,'Agincourt North':'Yorkdale-Glen Park']
income_data = transposed_df.transpose().reset_index()
income_data.columns = ["neighbourhood_name","median_income","average_income"]
income_data

neighbourhood_name  median_income  average_income
0                 Agincourt North        60162.0         73512.0
1    Agincourt South-Malvern West        59013.0         73156.0
2                       Alderwood        71585.0         85206.0
3                           Annex        57825.0        114333.0
4               Banbury-Don Mills        70039.0        115944.0
..                            ...            ...             ...
135                      Wychwood        55941.0         84199.0
136                Yonge-Eglinton        72977.0        126230.0
137                Yonge-St.Clair        71728.0        153993.0
138       York University Heights        45534.0         55531.0
139            Yorkdale-Glen Park        54093.0         72429.0

[140 rows x 3 columns]

In [5]:
neighbourhood = pd.read_csv('../Resources/Neighbourhoods.csv')

# For merge the DataFrames, add the neighbourhood_column
neighbourhood['neighbourhood_name'] = neighbourhood['AREA_NAME'].str.replace(' \(.+\)', '')
income_data['neighbourhood_name'] = income_data['neighbourhood_name'].str.replace(' \(.+\)', '')

In [6]:
income_neighbourhood = pd.merge(income_data, neighbourhood,  on='neighbourhood_name')
income = income_neighbourhood[['AREA_SHORT_CODE', 'neighbourhood_name', 'median_income', \
                               'average_income']]
income = income.rename(columns={'AREA_SHORT_CODE':'neighbourhood_id'})
income = income.set_index('neighbourhood_id')
income = income.sort_index()

## Final Transformed Table - income

In [7]:
income

neighbourhood_name  median_income  \
neighbourhood_id                                                     
1                            West Humber-Clairville        66241.0   
2                 Mount Olive-Silverstone-Jamestown        49934.0   
3                      Thistletown-Beaumond Heights        62042.0   
4                                   Rexdale-Kipling        56545.0   
5                                  Elms-Old Rexdale        50846.0   
...                                             ...            ...   
136                                       West Hill        49713.0   
137                                          Woburn        52018.0   
138                                   Eglinton East        46495.0   
139                             Scarborough Village        42131.0   
140                                       Guildwood        76055.0   

                  average_income  
neighbourhood_id                  
1                        76228.0  
2                        58605.0  
3                        73512.0  
4                        66781.0  
5                        63201.0  
...                          ...  
136                      63461.0  
137                      63651.0  
138                      58035.0  
139                      62141.0  
140                      96885.0  

[140 rows x 3 columns]

In [8]:
income.to_csv('../clean_data/income.csv')